In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/data/elastic-notebook/data/fashion-mnist'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/data/elastic-notebook/data/fashion-mnist/2020-mpcs53111-hw5-fashionmnist.zip
/data/elastic-notebook/data/fashion-mnist/fashionmnist_test.npy
/data/elastic-notebook/data/fashion-mnist/fashionmnist_train.npy


In [2]:
import numpy as np

dataset_prefix = "fashionmnist" 

train_data = np.load("/data/elastic-notebook/data/fashion-mnist/{}_train.npy".format(dataset_prefix))
test_data = np.load("/data/elastic-notebook/data/fashion-mnist/{}_test.npy".format(dataset_prefix))

train_images = train_data[:, :-1].reshape(-1, 1, 28, 28)
train_labels = train_data[:, -1]
test_images = test_data.reshape(-1, 1, 28, 28)/255

In [3]:
import torchvision
from PIL import Image

pil_images = [Image.fromarray((image.reshape(image.shape[1:])*255).astype("uint8"), 'L') for image in train_images]
def pil_to_np(images):
    images = np.array([np.array(image) for image in images])
    return images.reshape(-1, 1, 28, 28)/255.
modified_images = [torchvision.transforms.functional.hflip(image) for image in pil_images]
train_images = np.concatenate([train_images, pil_to_np(modified_images)], axis=0)
modified_images = [torchvision.transforms.functional.affine(image, 0, (-2, 0), 1, 0) for image in pil_images]
train_images = np.concatenate([train_images, pil_to_np(modified_images)], axis=0)
modified_images = [torchvision.transforms.functional.affine(image, 0, (+2, 0), 1, 0) for image in pil_images]
train_images = np.concatenate([train_images, pil_to_np(modified_images)], axis=0)
#modified_images = [torchvision.transforms.functional.affine(image, 0, (0, -2), 1, 0) for image in pil_images]
#train_images = np.concatenate([train_images, pil_to_np(modified_images)], axis=0)
#modified_images = [torchvision.transforms.functional.affine(image, 0, (0, +2), 1, 0) for image in pil_images]
#train_images = np.concatenate([train_images, pil_to_np(modified_images)], axis=0)
modified_images = None
pil_images = None
train_labels = np.concatenate([train_labels]*4, axis=0)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(p=0.2),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(p=0.2),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(p=0.2),
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(4608, 2048),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x1 = self.cnn_layers(x).view(x.shape[0], -1)
        return self.linear_layers(x1)

class Model:
    def fit(self, X, y, lr=0.001, n_epochs=50, test_data=None):
        self.net = Net()
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.net = self.net.to(self.device)
        self.loss = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.net.parameters(), lr=lr)
        train_data = (X, y)
        X = torch.from_numpy(X).float().to(self.device)
        y = torch.from_numpy(y).long().to(self.device)
        data = utils.data.TensorDataset(X, y)
        data_loader = utils.data.DataLoader(data, batch_size=512, shuffle=True)
        for ep in range(n_epochs):
            self.net.train()
            for batch_X, batch_y in data_loader:
                self.optimizer.zero_grad()
                y_hat = self.net(batch_X)
                loss = self.loss(y_hat, batch_y)
                loss.backward()
                self.optimizer.step()
            if ep % 1 == 0:
                print(f'Epoch: {ep}, Loss: {loss}')
                print(f'Train Accuracy: {self.score(*train_data)}')
                if test_data: print(f'Test Accuracy: {self.score(*test_data)}')
    def predict(self, X):
        with torch.no_grad():
            self.net.eval()
            X = torch.from_numpy(X).float().to(self.device)
            data = utils.data.TensorDataset(X)
            data_loader = utils.data.DataLoader(data, batch_size=512)
            outputs = []
            for batch, in data_loader:
                outputs.append(self.net(batch))
            out = torch.cat(outputs)
            return torch.argmax(out, axis=1)
    def score(self, X, y):
        y_hat = self.predict(X)
        y = torch.from_numpy(y).long().to(self.device)
        return int((y == y_hat).sum())/y.shape[0]
    def get_params(self, deep=True): return {}
    def set_params(self, **__): pass

In [5]:
model = Model()
model.fit(train_images, train_labels)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4063386/2550561381.py", line 2, in <module>
    model.fit(train_images, train_labels)
  File "/tmp/ipykernel_4063386/199982349.py", line 66, in fit
    loss.backward()
  File "/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/torch/_tensor.py", line 396, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/torch/autograd/__init__.py", line 173, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/local/data/elastic-noteboo

In [ ]:
out = model.predict(test_images)
import csv
with open(f'/data/elastic-notebook/tmp/{dataset_prefix}_out.csv', mode='w', newline='') as out_file:
    fieldnames = ['Id', 'Category']
    writer = csv.DictWriter(out_file, fieldnames=fieldnames)

    writer.writeheader()
    for i, j in enumerate(out):
        writer.writerow({'Id': i, 'Category': int(j)})